<a href="https://colab.research.google.com/github/cbarlow-corelationinc/troubled_loan_detection/blob/main/LendingRestoreModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Restoring a Saved Model#


<a id='introduction'></a>
## Introduction##
This is an example of loading an already trained model from disk and using it to make predictions.

<b>Table of Contents</b>
1. [Introduction](#introduction)
2. [Import Necessary Packages](#imports)
3. [Load and Normalize the Data](#load_data)
4. [Load the Model from and hdf5 Disk File](#load_model)
5. [Summarize the Training Results](#summarize)
6. [Conclusion](#conclusion)

<a id='imports'></a>
## Import Necessary Packages ##

First, let's run the cell below (put your cursor in the cell and press Shift-Enter) to import all the packages that you will need.
- [numpy](https://www.numpy.org/) is the fundamental package for scientific computing with Python.
- [keras](https://pypi.org/project/Keras/) is a high level neural network library for deep learning applications.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python. 
- [pandas](https://pandas.pydata.org/) is a data analysis library.
- [seaborn](https://seaborn.pydata.org/) is a data visualization library based on matplotlib.
- [scikitplot](https://anaconda.org/conda-forge/scikit-plot) is yet another plotting library.
- [sklearn](https://scikit-learn.org/stable/) is a collection of algorithms for classification, regression, and clustering.
- [h5py](http://www.h5py.org) is a common package to interact with a dataset that is stored on an H5 file.

<b><i>Only run the following cell if you are running on Google Colab environment!<i></b> This cell is intended to install any Python libraries that are missing from the default environment in Google Colab and to pull down the necessary utility Python code and data files from Git Hub.

In [ ]:
import sys

if sys.platform.startswith('linux'):
    # Extra imports for pulling zip file from Carl GitHub repository.
    import requests, zipfile, io

    # Install additional Python libraries that are missing from the default environment in Google Colab
    %pip install scikit-plot

    # Remove folders that may be left over from previous runs
    !rm -f datasets/*
    !rmdir datasets
    !rm -f helper_code/*
    !rmdir helper_code
    !rm -f models/*
    !rmdir models

    # Get necessary Python code and data files from Git Hub repository
    r = requests.get( 'https://raw.github.com/cbarlow-corelationinc/troubled_loan_detection/main/TroubledLoanDetectionFiles.zip')
    r.status_code
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    !ls -Abl

In [ ]:
# Import packages
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
from cb3_ann_utils import describe_datasets, build_compile_fit_model, \
    my_f1_score, cb3_plot_confusion_matrix, cb3_calculateResults1, cb3_calculateResults2, cb3_displayResults

print("Done importing required modules.")

<a id='load_data'></a>
## Load and Normalize the Data ##

Use pandas.read_csv() function to read the loan data from a CSV disk files into a Pandas DataFrame, and then shuffle and split up the data into a training set, a dev set (also known as a "cross validation" or a "holdout" set), and a test set.  Training is only done on the training set.  The dev set it used to evaluate the ability of the trained network to generalize well and can be used to help "tune" hyperparameter settings.  Finally, the test set is used to see if the performance is relatively consistent between the dev set and the test set, so that we can have confidence that the test set performance is an accurate representation of how well the network will perform on data that was not seen during training (assuming the training data was a good representative sample).  NOTE: THE FIELDS IN THE DATA FILE MUST MATCH 100% THE FIELDS THAT WERE USED AS FEATURES WHILE TRAINING THE MODEL THAT WE WILL USE!  Also, when running future predictions on unlabeled data, you will need to know the original mean and standard deviation that were used to normalize the training data so they can also be applied to new data.  So in a production product, save the mean and standard deviation of the training data along with the model itself.

In [ ]:
np.random.seed(31)    # in case you need to be able to duplicate exact shuffling of data

# Read CSV data
loan_data = pd.read_csv('datasets/AnonymousLoans20190930OregonStateFull.csv', index_col = 'LOAN_SERIAL')

# Extract and encode categorical features
ln_type_cat = loan_data[['LN_TYPE_CATEGORY']].astype(CategoricalDtype(["CC", "CE", "LC", "OE"]))
ln_type_cat = pd.get_dummies(ln_type_cat, prefix='LN_TYPE_CAT', drop_first=True)
ln_type_cat.head()

categorical_features = ln_type_cat

# Extract numerical features
#numerical_features = loan_data[['CREDIT_LIMIT', 'COBORROWER_COUNT', 'ORIGINAL_TERM_IN_MONTHS', 'BALANCE', 'MONTHLY_PMT', 'INTEREST_RATE', \
#                               'DQ_DAYS', 'STILL_MISSED_FIRST_PMT', 'HAS_COLLATERAL', 'AGE_IN_DAYS', 'MEMBERSHIP_DAYS', 'PMT_COUNT_MADE', \
#                               'PMT_COUNT_DQ_30_TO_59', 'PMT_COUNT_DQ_60_TO_89', 'PMT_COUNT_DQ_90_TO_119', \
#                               'PMT_COUNT_DQ_120_AND_UP', 'MONTHLY_INCOME', 'MONTHLY_EXPENSE', 'LR_AMOUNT', 'SCORE', 'AGE_OF_SCORE', \
#                               'AGGR_SHARE_COUNT', 'AGGR_SHARE_BAL', 'DD_CHANGE', 'CREDIT_LIMIT_INCREASE_COUNT', 'OFF_AUTO_PAY_COUNT']]
numerical_features = loan_data[['DQ_DAYS', 'AGGR_SHARE_BAL', 'PMT_COUNT_MADE', 'MONTHLY_PMT', 'PMT_COUNT_DQ_30_TO_59', \
                                'CREDIT_LIMIT', 'INTEREST_RATE', 'PMT_COUNT_DQ_90_TO_119', 'SCORE', \
                                'COBORROWER_COUNT', 'MONTHLY_EXPENSE']]
#loan_features = pd.concat([categorical_features, numerical_features], axis=1)
loan_features = numerical_features


loan_labels = loan_data['BINARY_STATUS']
frame = { 'BINARY_STATUS': loan_labels } 
loan_labels_df = pd.DataFrame(frame)

# Split the data up in train and dev sets
X_train, X_test, y_train, y_test = train_test_split(loan_features, loan_labels_df, train_size=0.8)
# Further split dev data into dev and test sets
X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.50)

# Normalize the data
train_mean = X_train.mean(axis=0)
train_std = X_train.std(axis=0)
train_std = np.where(train_std==0.0, 1.e-8, train_std) # avoid divide by zero error
X_train -= train_mean
X_train /= train_std
# scale dev data and test data based on train data mean and standard deviation!
X_dev -= train_mean
X_dev /= train_std
X_test -= train_mean
X_test /= train_std
    
n = X_train.shape[1]  # feature count
#describe_datasets(X_train, y_train, X_dev, y_dev, X_test, y_test, n)
describe_datasets(X_train, y_train, X_dev, y_dev, X_test, y_test, n)

<a id='load_model'></a>
## Load the Model from an hdf5 Disk File ##
The model has already been trained previously, so now you can just load the model from an hdf5 disk file, and then run the summary() function to display the neural network architecture used.

In [ ]:
# load model
model = load_model('./models/model_20200831_s31a.h5')
# summarize model.
model.summary()

<a id='summarize'></a>
## Summarize the Training Results ##
Display summary of various statistics about the performance of the model against the test data.

In [ ]:
# Now make predictions on test data
y_pred_test = model.predict(X_test)
y_pred_test[y_pred_test>=0.5] = 1
y_pred_test[y_pred_test<0.5] = 0
y_test_numpy = y_test.to_numpy()

(tp, tn, fp, fn) = cb3_calculateResults1(y_pred_test, y_test_numpy)
(accuracy, precision, recall, f1, cm) = cb3_calculateResults2(tp, tn, fp, fn)
cb3_displayResults(accuracy, precision, recall, f1, cm, "Test Set")

Here are what the results would look like if we just set p(badLoan)=0:

In [ ]:
bad = tp + fn
good = tn + fp
totLoans = bad + good
(accuracy, precision, recall, f1, cm) = cb3_calculateResults2(0, good, 0, bad)
cb3_displayResults(accuracy, precision, recall, f1, cm, "p(BadLoans)=0")

And here are what the results would look like if we just randomly labeled a small number as bad (p(badLoan) = actual percentage bad) in hopes of improving accuracy from the p(badLoan) = 0 model:

In [ ]:
p = bad
n = good
tp = int(round(p * bad / totLoans))
fp = p - tp
tn = int(round(n * good / totLoans))
fn = n - tn
(accuracy, precision, recall, f1, cm) = cb3_calculateResults2(tp, tn, fp, fn)
cb3_displayResults(accuracy, precision, recall, f1, cm, "p(BadLoans)=observedProbability")

You can also look at roughly which input features had the most-to-least influence on predictions (take this simplistic interpretation of the weights in the first hidden layer with a grain of salt):

In [ ]:
weights = np.sum(np.absolute(model.layers[0].get_weights()[0]), axis=1)
print("Weights in original feature order:")
print(weights)
wi = np.argsort(weights)
ws = sorted(weights)
print("\nWeights sorted most to least influential:")
for i in range(len(wi)-1, -1, -1):
    print(ws[i],X_train.columns.values[wi[i]])

We can also go back and look at a few examples of false negatives (where we predicted a good loan in the dev data, but it was actually bad), getting the full, non-normalized data from original_data. Then we can go back to the credit union and find these loans and examine them for further "inspiration" on what additional features might have made this more predictable. 

In [ ]:
# Now look at up to three examples of false negatives for inspiration
false_negative_tuple = np.where(((y_pred_test==0) & (y_test_numpy==1)))
false_negative_array = false_negative_tuple[0]
l = len(false_negative_array)
if l>0:
    print("\nExamples of false negatives in test data:")
for i in range(min(l, 3)):
    loan_serial = X_test.index.values[false_negative_array[i]]
    print("\nLoan Serial ",loan_serial)
    print(loan_data.loc[loan_serial])

Or we can list the serial numbers of <i>all</i> false negatives:

In [ ]:
if l>0:
    print("\nFalse negatives in test data:")
for i in range(l):
    loan_serial = X_test.index.values[false_negative_array[i]]
    print("Loan Serial ",loan_serial)
    #print(loan_data.loc[loan_serial])

<a id='conclusion'></a>
## Conclusion  ##
It takes much less time to make predictions using an existing, trained model, than it does to train a model in the first place.


[Return to top of Notebook](#introduction)